## Important note :  Pre-requisite


#### This Notebook requires an existing Knowldge base on bedrock .To create a knowldge base you can execute the code provided in : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/02_KnowledgeBases_and_RAG/0_create_ingest_documents_test_kb.ipynb

# Automated Document Processing Pipeline: Using Foundation Models for Smart Document Chunking and Knowledge Base Integration

This notebook demonstrates an end-to-end automated solution for intelligent document processing using Foundation Models (FM). The pipeline performs three key functions:


#### Notebook Walkthrough
The pipeline streamlines the entire process from document analysis to knowledge base population, making it efficient to prepare documents for advanced query and retrieval operations.

##### 1.Document Structure Analysis: FM Automatically analyzes document structure and content to determine the optimal chunking strategy
##### 2. Configuration Generation: Creates customized chunking parameters based on the analysis

##### 3.Knowledge Base Integration: Processes and loads the chunked documents into a Bedrock Knowledge Base powered by OpenSearch Serverless

![data_ingestion](./img/chunkingAdvs.jpg)

#### Steps: 

1. Setup Access and Permissions:
        Create Amazon Bedrock Knowledge Base execution role
        Configure necessary IAM policies for:
            S3 data access
            OpenSearch Serverless writing permissions
        Reference template available in "0_create_ingest_documents_test_kb" notebook

2. Document Analysis Using Claude Sonnet:
        Process files within target folder.For each document, Claude analyzes and recommends:
            Optimal chunking strategy (FIXED_SIZE/NONE/HIERARCHICAL/SEMANTIC)
            Specific configuration parameters
            Custom processing requirements

3. Data Preparation and Storage:
        Upload analyzed files to designated S3 buckets
        Configure buckets as data source for Bedrock KB DataStore

4. Ingestion Process:
        Initiate ingestion job via Knowledge Base APIs
5. Validation:
        Test ingestion completion
        Verify data accessibility and accuracy

#### Pre-requisites

This notebook requires permissions to:

1. Create and delete Amazon IAM roles
2. Create, update and delete Amazon S3 buckets
3. Access Amazon Bedrock
4. Bedrock roles to access s3 buckets (3 bucket)
5. Access to Amazon OpenSearch Serverless

If running on SageMaker Studio, you should add the following managed policies to your role:

- IAMFullAccess
- AWSLambda_FullAccess
- AmazonS3FullAccess-


### Install required libraries

In [1]:
%pip install --force-reinstall -q -r ./requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
aiobotocore 2.13.3 requires botocore<1.34.163,>=1.34.70, but you have botocore 1.35.90 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.1 which is incompatible.
autogluon-core 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-core 1.1.1 requires scipy<1.13,>=1.5.4, but you have scipy 1.14.1 which is incompatible.
autogluon-features 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 whi

In [1]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [2]:
import boto3

import json

# create a boto3 session to dynamically get and set the region name
session = boto3.Session() 

AWS_REGION = session.region_name
bedrock = boto3.client('bedrock-runtime',region_name=AWS_REGION)
bedrock_agent_client = session.client('bedrock-agent', region_name=AWS_REGION)

MODEL_NAME = "anthropic.claude-3-5-sonnet-20241022-v2:0" 
datasources =[]
#create a folder data if not yet done and 
path="data"
# To get knowledgeBaseId look int Amazon Bedrock > knowledgeBaseId > knowledgeBaseId
# This ID should bave being created from first notebook
# https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/02_KnowledgeBases_and_RAG/0_create_ingest_documents_test_kb.ipynb

kb_id= "XXXXX" # Retrieve KB First 
kb =  bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id)
print(kb)

{'ResponseMetadata': {'RequestId': 'c17d25dc-bf21-4d92-a638-1bbfb146074f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 31 Dec 2024 07:51:04 GMT', 'content-type': 'application/json', 'content-length': '928', 'connection': 'keep-alive', 'x-amzn-requestid': 'c17d25dc-bf21-4d92-a638-1bbfb146074f', 'x-amz-apigw-id': 'DpZQWHw3PHcEaig=', 'x-amzn-trace-id': 'Root=1-6773a268-4cd5639654aa10be09575d02'}, 'RetryAttempts': 0}, 'knowledgeBase': {'createdAt': datetime.datetime(2024, 12, 21, 19, 45, 35, 398774, tzinfo=tzlocal()), 'description': 'Amazon shareholder letter knowledge base.', 'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:305810904819:knowledge-base/QTAWVUUY9G', 'knowledgeBaseConfiguration': {'type': 'VECTOR', 'vectorKnowledgeBaseConfiguration': {'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1'}}, 'knowledgeBaseId': 'QTAWVUUY9G', 'name': 'bedrock-sample-knowledge-base-410', 'roleArn': 'arn:aws:iam::305810904819:role/AmazonBedrockExecut

In [3]:
#bedrock_kb_execution_role = kb['roleArn']
bedrock_kb_execution_role_arn = kb ['knowledgeBase']['roleArn']
bedrock_kb_execution_role=  bedrock_kb_execution_role_arn.split('/')[-1]
print  (bedrock_kb_execution_role_arn)
print  (bedrock_kb_execution_role)

arn:aws:iam::305810904819:role/AmazonBedrockExecutionRoleForKnowledgeBase_504
AmazonBedrockExecutionRoleForKnowledgeBase_504


### Supporting functions
##### Createbucket: Checks if an S3 bucket exists and creates it if it doesn't. 
##### Upload_file: Upload_files to bucket: Upload a file to an S3 bucket
##### Listfile: List all files in a specified directory:

In [4]:
import logging
import boto3
from botocore.exceptions import ClientError
import os

def createbucket(bucketname):
    """
    Checks if an S3 bucket exists and creates it if it doesn't. 
    Args:
        bucket_name (str): Name of the S3 bucket to create (must be globally unique)
    Raises:
        ClientError: If there's an error accessing or creating the bucket
    """
    try:
        s3_client = boto3.client('s3')
        s3_client.head_bucket(Bucket=bucketname)
        print(f'Bucket {bucketname} Exists')
    except ClientError as e:
        print(f'Creating bucket {bucketname}')
        if AWS_REGION  == "us-east-1":
            s3bucket = s3_client.create_bucket(
                Bucket=bucketname)
        else:
            s3bucket = s3_client.create_bucket(
            Bucket=bucketname,
            CreateBucketConfiguration={ 'LocationConstraint': AWS_REGION }
        )

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def listfile (folder):
    """
    List all files in a specified directory.
    
    Args:
        folder (str): Path to the directory to list files from
    
    Returns:
        list: A list of filenames in the specified directory
        """
    dir_list = os.listdir(folder)
    return dir_list
import boto3
from botocore.exceptions import ClientError
import logging

def delete_bucket_and_objects(bucket_name):
    # Create an S3 client
    s3_client = boto3.client('s3')
    # Create an S3 resource
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    bucket.objects.all().delete()
    # Delete the bucket itself
    bucket.delete()

#### Download and prepare dataset


In [5]:
# if not yet created, create folder already
#!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://www.apple.com/newsroom/pdfs/Q3FY18ConsolidatedFinancialStatements.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'Q3FY18ConsolidatedFinancialStatements.pdf'
]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

#### Create 3 S3 buckets for 3 data sources 
##### Check if bucket exists, and if not create S3 bucket by knowledge base data source,  each bucket will be used to load files with corrspending strategy :semantic, fixed_size , HIERARCHICAl

In [6]:
import random
suffix = random.randrange(200, 900)
s3_client = boto3.client('s3')
bucket_name_semantic = 'kb-dataset-bucket-semantic-' + str(suffix)                                                        #### Provide your bucket name which is already created
bucket_name_fixed = 'kb-dataset-bucket-fixed-'+str(suffix)  
bucket_name_hierachical='kb-dataset-bucket-hierarchical-'+str(suffix) 
s3_policy_name = 'AmazonBedrockS3PolicyForKnowledgeBase_'+ str(suffix)

createbucket(bucket_name_semantic)
createbucket(bucket_name_fixed)
createbucket(bucket_name_hierachical)


Creating bucket kb-dataset-bucket-semantic-422
Creating bucket kb-dataset-bucket-fixed-422
Creating bucket kb-dataset-bucket-hierarchical-422


##### Create read and list S3 policy on the 3 buckets and attach it to existing Bedrock role "bedrock_kb_execution_role"


In [7]:
account_number = boto3.client('sts').get_caller_identity().get('Account')
iam_client = session.client('iam')
s3_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:ListBucket"
                ],
                "Resource": [
                    f"arn:aws:s3:::{bucket_name_semantic}",
                    f"arn:aws:s3:::{bucket_name_semantic}/*", 
                    f"arn:aws:s3:::{bucket_name_fixed}",
                    f"arn:aws:s3:::{bucket_name_fixed}/*", 
                    f"arn:aws:s3:::{bucket_name_hierachical}",
                    f"arn:aws:s3:::{bucket_name_hierachical}/*"
                ],
                "Condition": {
                    "StringEquals": {
                        "aws:ResourceAccount": f"{account_number}"
                    }
                }
            }
        ]
}
s3_policy = iam_client.create_policy(
            PolicyName=s3_policy_name,
            PolicyDocument=json.dumps(s3_policy_document),
            Description='Policy for reading documents from s3')

        # fetch arn of this policy 
s3_policy_arn = s3_policy["Policy"]["Arn"]
iam_client = session.client('iam')
fm_policy_arn = f"arn:aws:iam::{account_number}:policy/{s3_policy_name}"
iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role,
        PolicyArn=fm_policy_arn
    )

{'ResponseMetadata': {'RequestId': '72fa0faf-7f08-4488-a0dd-a51a6ec3a1b9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 31 Dec 2024 07:51:23 GMT',
   'x-amzn-requestid': '72fa0faf-7f08-4488-a0dd-a51a6ec3a1b9',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

##### Sends a prompt to Claude Sonnet  via Amazon Bedrock and returns the generated response.
    

In [16]:
def get_completion(prompt):
    """
    Sends a prompt to Claude Sonnet via Amazon Bedrock and returns the generated response.
    Args:
        prompt (str): The input text prompt to send to the AI model.

    Returns:
        str: The generated text response """

    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": ''
        }
    )
    response = bedrock.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())
    return response_body.get('content')[0].get('text')

 #### Chunkingadvise function
 ##### Analyzes a PDF document and recommends optimal LLM chunking strategy with parameters. This function loads a PDF file, analyzes its content using LLM, and provides  recommendations for chunking strategy (FIXED_SIZE, NONE, HIERARCHICAL, or SEMANTIC) along with specific configuration parameters.
   - Args:
        - file (str): Name of the PDF file located in the 'data' directory
        -  Returns:
            - dict: JSON containing recommended chunking strategy and parameters:
              For HIERARCHICAL:
                - Recommend only one Strategy
                - Maximum Parent chunk token size
                - Maximum child chunk token size
                - Overlap Tokens
                - Rational
            - For SEMANTIC:
                - Recommend only one Strategy
                - Maximum tokens
                - Buffer size
                - Breakpoint percentile threshold
                - Rational:
            - For FIXED-SIZE
              -  overlapPercentage'
              -  parsed_data['maxTokens'


In [17]:
def Chunkingadvise (file):
    """
    Analyzes a PDF document and recommends optimal LLM chunking strategy with parameters.

    This function loads a PDF file, analyzes its content using LLM, and provides 
    recommendations for chunking strategy (FIXED_SIZE, NONE, HIERARCHICAL, or SEMANTIC)
    along with specific configuration parameters.
    Args:
        file (str): Name of the PDF file located in the 'data' directory
    Returns:
        dict: JSON containing recommended chunking strategy and parameters:
            For HIERARCHICAL:
                - Recommend only one Strategy
                - Maximum Parent chunk token size
                - Maximum child chunk token size
                - Overlap Tokens
                - Rational
            For SEMANTIC:
                - Recommend only one Strategy
                - Maximum tokens
                - Buffer size
                - Breakpoint percentile threshold
                - Rational
  """
    my_docs = []
    my_strategies =[]
    strategy=""
    strategytext=""
    path="data"
    strategylist =[]
    metadata = [
        dict(year=2023, source=file)]
    from langchain.document_loaders import PyPDFLoader
    file = path +"/"+ file
    loader = PyPDFLoader(file)
    document = loader.load()
    loader = PyPDFLoader(file)
   # print ("path + file :: ", file)
    document = loader.load()
   # print ("path + file :: ", document)
   
    prompt = f"""SYSTEM you are an advisor expert in LLM chunking strategies,
    USER can you analyze the type,content, format, structure and size of {document}. 
    Can you advise on best LLM chunking Strategy based on this analysis. Recommend only one Strategy, however show recommended strategy prefernece ratio 
    Available strategies to recommend from are : FIXED_SIZE or NONE or HIERARCHICAL or SEMANTIC
    Decide on recommendatin first and then , what is the recommendation?   """
    res = get_completion(prompt)
    print(res)
    prompt = f""" USER based on recommnedation provide in {res} 
    if you recommend HIERARCHICAL chunking then provide recommendation for: 
    Parent: Maximum parent chunk token size. 
    Child: Maximum child chunk token size and Overlap Tokens: Number of overlap tokens between each parent chunk and between each parent and its children.
    If recommendation is HIERARCHICAL then provide response using JSON format
    with the keys as \"Recommend only one Strategy\", \"Maximum Parent chunk token size\", \"Maximum child chunk token size\",\"Overlap Tokens\", 
    \"Rational:please explain rational for decision and explain why each other choice is not prefered, keep reational to 100 words maximum. \" . provide crisp and clear answer, 
    if you recommend  SEMANTIC then provide response using  JSON format with
    the keys as \"Recommend only one Strategy\",\" Maximum tokens\", \"Buffer size\",\"Breakpoint percentile threshold\", 
    Buffer size should be less or equal than 1 , Breakpoint percentile threshold should >= 50
    \"Rational:please explain rational for decision and explain why each other choice is not prefered, keep reational to 100 words maximum. \" . provide crisp and clear answer, 
    do not provide recommendation if not enough data inputs and say sorry I need more data,
    if you recommend  FIXED_SIZE then provide response using  JSON format with
    the keys as \"Recommend only one Strategy\",\" maxTokens\", \"overlapPercentage \",
    \"Rational:please explain rational for decision and explain why each other choice is not prefered, keep reational to 100 words maximum. \" . provide crisp and clear answer, 
    do not provide recommendation if not enough data inputs and say sorry I need more data"""
    res = get_completion(prompt)
    print(res)
    parsed_data = json.loads(res )
    return parsed_data

#### ingestbystrategy: function that configure chunking strategy parameters for Bedrock Knowledge Base ingestion based on recommended strategy.
   - Args:
       - parsed_data (dict): Dictionary containing chunking strategy recommendation and parameters
    - Returns:
        - tuple: Contains:
            - chunking_strategy_config (dict): Configuration for the chosen chunking strategy
            - bucket_name (str): S3 bucket name for storage
            - name (str): Knowledge base name
            - description (str): Knowledge base description 
            - s3_configuration (dict): S3 configuration with bucket ARN


In [18]:
def ingestbystrategy(parsed_data):
    """
    Configures chunking strategy parameters for Bedrock Knowledge Base ingestion based on recommended strategy.

    Args:
        parsed_data (dict): Dictionary containing chunking strategy recommendation and parameters

    Returns:
        tuple: Contains:
            - chunking_strategy_config (dict): Configuration for the chosen chunking strategy
            - bucket_name (str): S3 bucket name for storage
            - name (str): Knowledge base name
            - description (str): Knowledge base description 
            - s3_configuration (dict): S3 configuration with bucket ARN

    Example:
        >>> strategy_config, bucket, kb_name, desc, s3_config = ingest_by_strategy(strategy_data)
    """
    chunkingStrategyConfiguration ={}
    # print("Strategy::", parsed_data)
    strategy= parsed_data['Recommend only one Strategy']

    if strategy =='HIERARCHICAL':
       p1 = parsed_data['Maximum Parent chunk token size']
       p2= parsed_data['Maximum child chunk token size']
       p3= parsed_data['Overlap Tokens'] 
       bucket_name=bucket_name_hierachical 
       name = f"bedrock-sample-knowledge-base-HIERARCHICAL"
       description = "Bedrock Knowledge Bases for and S3 HIERARCHICAL"
    # HIERARCHICAL Chunking
       chunkingStrategyConfiguration = {
                                            "chunkingStrategy": "HIERARCHICAL",      
                                            "hierarchicalChunkingConfiguration": {  
                                                                                    'levelConfigurations': [
                                                                                        {
                                                                                            'maxTokens': p1
                                                                                        },
                                                                                        {
                                                                                            'maxTokens': p2
                                                                                        }
                                                                                    ],
                                                                                    'overlapTokens': p3
                                                                                }
                                        }
    
    # # SEMANTIC Chunking 
    if strategy =='SEMANTIC':
        p3 = parsed_data['Maximum tokens']
        p2= int(parsed_data['Buffer size'])
        p1= parsed_data['Breakpoint percentile threshold']
        bucket_name= bucket_name_semantic
        name = f"bedrock-sample-knowledge-base-SEMANTIC"
        description = "Bedrock Knowledge Bases for and S3 SEMANTIC"
        chunkingStrategyConfiguration = { "chunkingStrategy": "SEMANTIC",
                                         "semanticChunkingConfiguration": {          
                                                                              'breakpointPercentileThreshold': p1,
                                                                              'bufferSize': p2,
                                                                              'maxTokens': p3
                                                                        }
                                    }


    if strategy =='FIXED_SIZE':
        p2= int(parsed_data['overlapPercentage'])
        p1= int (parsed_data['maxTokens'])
        bucket_name=bucket_name_fixed
        name = f"bedrock-sample-knowledge-base-FIXED"
        description = "Bedrock Knowledge Bases for and S3 FIXED"
      
        chunkingStrategyConfiguration = { "chunkingStrategy": "FIXED_SIZE",
                                         "semanticChunkingConfiguration": {          
                                                                              "maxTokens": p1,
                                                                              "overlapPercentage":p2
                                                                     
                                                                        }
                                    }
    
    s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
    } 
    return chunkingStrategyConfiguration ,bucket_name , name , description ,s3Configuration 

#### Function to Creates or retrieves a data source in an Amazon Bedrock Knowledge Base
 
#### First checks if a data source with the given name exists. If found, returns the existing  data source. Otherwise creates a new one with specified configurations.
- Args:
    - name (str): Name of the data source
    - description (str): Description of the data source
    - knowledge_base_id (str): ID of the knowledge base to create data source in
    - s3_configuration (dict): S3 bucket configuration for the data source
    - chunking_strategy_configuration (dict): Configuration for text chunking strategy
- Returns:
    - dict: Response containing the data source details from Bedrock

In [19]:
def createDS (name, description,knowledgeBaseId , s3Configuration , chunkingStrategyConfiguration ):
    """
    Creates or retrieves a data source in an Amazon Bedrock Knowledge Base.
    
    First checks if a data source with the given name exists. If found, returns the existing 
    data source. Otherwise creates a new one with specified configurations.

    Args:
        name (str): Name of the data source
        description (str): Description of the data source
        knowledge_base_id (str): ID of the knowledge base to create data source in
        s3_configuration (dict): S3 bucket configuration for the data source
        chunking_strategy_configuration (dict): Configuration for text chunking strategy

    Returns:
        dict: Response containing the data source details from Bedrock

    Raises:
        ClientError: If there's an error accessing or creating the data source
    """
    response = bedrock_agent_client.list_data_sources(
            knowledgeBaseId=kb_id,
            maxResults=12
        )
    for  i in range (len(response["dataSourceSummaries"])):
            print (response["dataSourceSummaries"][i] ["name"] ,"::", name)
            print (response["dataSourceSummaries"][i]["dataSourceId"])
            if response["dataSourceSummaries"][i] ["name"] == name:
                ds =  bedrock_agent_client.get_data_source(knowledgeBaseId = knowledgeBaseId, dataSourceId = response["dataSourceSummaries"][i-1]["dataSourceId"]  )
                return ds
           
    ds  = bedrock_agent_client.create_data_source(
                                                                        name = name,
                                                                        description = description,
                                                                        knowledgeBaseId = knowledgeBaseId,
                                                                        dataDeletionPolicy = 'DELETE',
                                                                        dataSourceConfiguration = {
                                                                            # # For S3 
                                                                            "type": "S3",
                                                                            "s3Configuration" : s3Configuration
                                                                            # # For Web URL 
                                                                            # "type": "WEB",
                                                                            # "webConfiguration":webConfiguration                                                                    
                                                                        },
                                                                        vectorIngestionConfiguration = {
                                                                            "chunkingConfiguration": chunkingStrategyConfiguration
                                                                        })
        
    
    return ds

### Process PDF files by analyzing content, creating data sources, and uploading to S3.

#### Workflow:
1. Lists all files in specified directory
2. For each PDF:
        - Analyzes for optimal chunking strategy
        - Creates data source with recommended configuration
        - Uploads file to appropriate S3 bucket 
<h5>

In [21]:
s3_client = boto3.client('s3')
dir_list1= listfile ("data")
print(dir_list1)
strategylist= []
for file in dir_list1:
    print (" print(f)" , file)
    if ".pdf" in file:
        chunkingStrategyConfiguration=[]
        strategy = Chunkingadvise (file)
        chunkingStrategyConfiguration ,bucket_name , name , description ,s3Configuration  = ingestbystrategy(strategy)
        print ("name", name)
        datasources = createDS (name, description,kb_id, s3Configuration , chunkingStrategyConfiguration )
        print (datasources)
        #ds_id = datasources[0]["dataSource"]["dataSourceId"]
        with open( path +"/"+ file, "rb") as f:
            print(bucket_name)
            print(f)
            s3_client.upload_fileobj(f, bucket_name, file)
       #print (strategylist)  

['.ipynb_checkpoints', 'AMZN-2022-Shareholder-Letter.pdf', 'AMZN-2021-Shareholder-Letter.pdf', 'Q3FY18ConsolidatedFinancialStatements.pdf']
 print(f) .ipynb_checkpoints
 print(f) AMZN-2022-Shareholder-Letter.pdf
Based on analyzing these documents, let me recommend a chunking strategy:

RECOMMENDED STRATEGY: SEMANTIC

Preference Ratio:
- SEMANTIC: 70%
- HIERARCHICAL: 20%
- FIXED_SIZE: 8%
- NONE: 2%

Reasoning for SEMANTIC recommendation:

1. Content Type Analysis:
- The document is a shareholder letter with clear thematic sections
- Contains distinct topic transitions and subject matter boundaries
- Has natural semantic groupings around business units, strategies, and updates

2. Structure Analysis:
- Document has clear paragraph-level organization
- Contains meaningful section breaks
- Topics are self-contained but interrelated

3. Format Analysis:
- Consistent formatting with clear paragraph breaks
- Mixed content including narrative text and bullet points
- Natural semantic boundarie

Ignoring wrong pointing object 8 0 (offset 0)


{
    "Recommend only one Strategy": "SEMANTIC",
    "Maximum tokens": 1000,
    "Buffer size": 0.8,
    "Breakpoint percentile threshold": 75,
    "Rational": "SEMANTIC chunking is optimal for this corporate letter due to its natural topic transitions and clear semantic boundaries between sections. The content's narrative flow and contextual relationships need to be preserved. HIERARCHICAL would be suboptimal as the document's structure isn't deeply nested. FIXED_SIZE would arbitrarily break coherent topics. NONE would make the content too large to process effectively. The chosen parameters allow for flexible chunk sizes while maintaining topic integrity and context."
}
name bedrock-sample-knowledge-base-SEMANTIC
bedrock-sample-knowledge-base-SEMANTIC :: bedrock-sample-knowledge-base-SEMANTIC
WXBJWYU11D
{'ResponseMetadata': {'RequestId': '27b11ba7-4ee5-40fe-b328-8fcc11d0b9e5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 31 Dec 2024 08:04:22 GMT', 'content-type': 'application/

Ignoring wrong pointing object 8 0 (offset 0)


Let me analyze the documents and recommend a chunking strategy:

Analysis:
1. Type: Financial statements (structured tabular data)
2. Content: Numerical data, financial metrics, organized in standard accounting format
3. Format: Consistent tabular layout with clear sections
4. Structure: 
   - Hierarchical organization (main statements → sections → line items)
   - Clear semantic relationships between numbers
   - Natural groupings (assets/liabilities, income/expenses, etc.)
5. Size: Moderate (~3 pages of dense financial data)

Strategy Preference Ratio:
HIERARCHICAL: 70%
SEMANTIC: 20%
FIXED_SIZE: 8%
NONE: 2%

Recommendation: HIERARCHICAL

Reasoning:
1. Financial statements have a natural hierarchical structure (statements → sections → subsections → line items)
2. Each statement (Income, Balance Sheet, Cash Flow) forms a logical hierarchy
3. Preserves the relationship between parent totals and child components
4. Maintains the context of financial metrics within their respective statem

#### Starts Ingestin  and monitors ingestion jobs for all data sources in a knowledge base.

In [22]:
from datetime import datetime    
import time
"""
    Starts and monitors ingestion jobs for all data sources in a knowledge base.
"""
sources  = bedrock_agent_client.list_data_sources(knowledgeBaseId = kb_id  )
for i in  range(len(sources["dataSourceSummaries"])):
    print ("ds [dataSourceId]", sources["dataSourceSummaries"] [i-1] ["dataSourceId"])
    start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb_id, dataSourceId = sources["dataSourceSummaries"] [i-1] ["dataSourceId"])
    job = start_job_response["ingestionJob"]
    print (job)
       # Get job 
    while(job['status']!='COMPLETE' ):
        get_job_response = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId = kb_id, dataSourceId = sources["dataSourceSummaries"][i-1] ["dataSourceId"], ingestionJobId = job["ingestionJobId"]
        )
        job = get_job_response["ingestionJob"]
    time.sleep(10)

ds [dataSourceId] WXBJWYU11D
{'dataSourceId': 'WXBJWYU11D', 'ingestionJobId': 'XFZYESD1KE', 'knowledgeBaseId': 'QTAWVUUY9G', 'startedAt': datetime.datetime(2024, 12, 31, 8, 5, 39, 476885, tzinfo=tzlocal()), 'statistics': {'numberOfDocumentsDeleted': 0, 'numberOfDocumentsFailed': 0, 'numberOfDocumentsScanned': 0, 'numberOfMetadataDocumentsModified': 0, 'numberOfMetadataDocumentsScanned': 0, 'numberOfModifiedDocumentsIndexed': 0, 'numberOfNewDocumentsIndexed': 0}, 'status': 'STARTING', 'updatedAt': datetime.datetime(2024, 12, 31, 8, 5, 39, 476885, tzinfo=tzlocal())}
ds [dataSourceId] IVAXJWS33Q
{'dataSourceId': 'IVAXJWS33Q', 'ingestionJobId': 'CGMPTICGX6', 'knowledgeBaseId': 'QTAWVUUY9G', 'startedAt': datetime.datetime(2024, 12, 31, 8, 6, 7, 38280, tzinfo=tzlocal()), 'statistics': {'numberOfDocumentsDeleted': 0, 'numberOfDocumentsFailed': 0, 'numberOfDocumentsScanned': 0, 'numberOfMetadataDocumentsModified': 0, 'numberOfMetadataDocumentsScanned': 0, 'numberOfModifiedDocumentsIndexed': 0,

#### Try out KB using RetrieveAndGenerate API

In [25]:
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"                                            # <Change it to any model of your choice which is supported by KB>
model_arn = f'arn:aws:bedrock:us-west-2::foundation-model/{model_id}'
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=AWS_REGION)
# ucomment to test 
#query = "what is AWS annualized revenue run rate"
query = "what is iphone sales in 2018"
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn
        }
    },
)

generated_text = response['output']['text']

print(generated_text)

I cannot determine the specific iPhone sales numbers for 2018 from the provided search results. While there is financial data from Apple Inc. for 2018 showing overall net sales of $53,265 million for the three months ended June 30, 2018, and $202,695 million for the nine months ended June 30, 2018, the results do not break down sales specifically for iPhones.


##### Clean buckets 
#####  NOTE : please delete also Bedrock  KB if not required by other works and data sources 


In [26]:

delete_bucket_and_objects(bucket_name_semantic)
delete_bucket_and_objects(bucket_name_fixed)
delete_bucket_and_objects(bucket_name_hierachical)

#### Conclusion: 

This notebook demonstrates an experimental approach using Foundation Models to determine optimal chunking strategies for different document types. While showing promising initial results, the current methodology is exploratory and requires further refinement.

#### Disclaimer:

Recommendations are based on base Foundation Models without fine-tuning
Results, validation against ground truth data would be required to validate results accurracy 

#### Proposed Next Steps:

1.Model Enhancement
        Implement fine-tuning on domain-specific data
        Experiment with different prompt engineering mechanisms

2.Validation Framework
        Establish ground truth dataset for testing
        Develop evaluation metrics for chunking quality
        Create a systematic testing methodology
